In [1]:
!pip install kagglehub


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download(
    "vjchoudhary7/customer-segmentation-tutorial-in-python"
)

/Users/dmitry/PycharmProjects/LinRegPractice/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path

'/Users/dmitry/.cache/kagglehub/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python/versions/1'

In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
)

In [5]:
os.listdir(path)

['Mall_Customers.csv']

In [ ]:
df = pd.read_csv(f"{path}/Mall_Customers.csv")

In [7]:
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [9]:
df.describe()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100)
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


In [ ]:
df_edited = df.assign(isMale=df["Gender"] == "Male")
df_edited = df_edited.drop("Gender", axis=1)
df_edited["isMale"] = df_edited["isMale"].astype(int)

In [31]:
df_edited.head()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100),isMale
0,1,19,15,39,1
1,2,21,15,81,1
2,3,20,16,6,0
3,4,23,16,77,0
4,5,31,17,40,0


In [11]:
def euqlid(point1, point2):
    return sum((x - y) ** 2 for x, y in zip(point1, point2))

In [12]:
k = 3

In [13]:
df_edited.min().values

array([ 1, 18, 15,  1,  0])

In [ ]:
lower_bound = df_edited.min(axis=0)
upper_bound = df_edited.max(axis=0)
centers = np.asarray(
    [np.random.uniform(low=lower_bound, high=upper_bound) for _ in range(k)]
)

In [15]:
centers

array([[1.48278627e+02, 3.47695394e+01, 5.95231088e+01, 7.53257529e+01,
        2.44471517e-01],
       [1.49146887e+02, 2.16123509e+01, 3.40987188e+01, 4.92930738e+01,
        3.31053572e-02],
       [1.38959941e+02, 3.80526612e+01, 1.29710464e+02, 7.77846640e+00,
        2.99758346e-01]])

In [16]:
df_edited = df_edited.values

In [17]:
df_edited

array([[  1,  19,  15,  39,   1],
       [  2,  21,  15,  81,   1],
       [  3,  20,  16,   6,   0],
       [  4,  23,  16,  77,   0],
       [  5,  31,  17,  40,   0],
       [  6,  22,  17,  76,   0],
       [  7,  35,  18,   6,   0],
       [  8,  23,  18,  94,   0],
       [  9,  64,  19,   3,   1],
       [ 10,  30,  19,  72,   0],
       [ 11,  67,  19,  14,   1],
       [ 12,  35,  19,  99,   0],
       [ 13,  58,  20,  15,   0],
       [ 14,  24,  20,  77,   0],
       [ 15,  37,  20,  13,   1],
       [ 16,  22,  20,  79,   1],
       [ 17,  35,  21,  35,   0],
       [ 18,  20,  21,  66,   1],
       [ 19,  52,  23,  29,   1],
       [ 20,  35,  23,  98,   0],
       [ 21,  35,  24,  35,   1],
       [ 22,  25,  24,  73,   1],
       [ 23,  46,  25,   5,   0],
       [ 24,  31,  25,  73,   1],
       [ 25,  54,  28,  14,   0],
       [ 26,  29,  28,  82,   1],
       [ 27,  45,  28,  32,   0],
       [ 28,  35,  28,  61,   1],
       [ 29,  40,  29,  31,   0],
       [ 30,  

In [18]:
def gen_center(group):
    return np.sum(group, axis=0) / group.shape[0]

In [19]:
gen_center(df_edited)

array([100.5 ,  38.85,  60.56,  50.2 ,   0.44])

In [20]:
df_edited.min(axis=0)

array([ 1, 18, 15,  1,  0])

# K-Means

In [ ]:
def KMeans(
    df: pd.DataFrame,
    k: int,
    centers=None,
    delta: float = 1e-6,
    iteration: int = 0,
    iterations_limit: int = 10,
):
    """
    :param df: - dataframe with objects
    :param centers: - cluster centers
    :param k: number of clusters
    :return: - cluster labels

    This functions is Kmeans-based clustering algorithm.
    """
    if centers is None:
        lower_bound = df_edited.min(axis=0)
        upper_bound = df_edited.max(axis=0)
        centers = np.asarray(
            [
                np.random.uniform(low=lower_bound, high=upper_bound)
                for _ in range(k)
            ]
        )

    labels = np.argmin(
        np.sqrt(np.sum((df[:, None, :] - centers[None, :, :]) ** 2, axis=2)),
        axis=1,
    )

    groups = [df[np.where(labels == i)] for i in range(k)]
    new_centers = np.asarray([gen_center(gr) for gr in groups])

    if np.all(new_centers - centers <= delta):
        return groups, new_centers, labels

    if iteration >= iterations_limit:
        return groups, new_centers, labels

    return KMeans(df, k, new_centers, delta, iteration + 1, iterations_limit)

In [53]:
groups, centers, lables = KMeans(df_edited, k)

In [27]:
centers[0, :][0]

np.float64(33.01538461538462)

In [54]:
lables

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [ ]:
fig = go.Figure()

fig.update_layout(
    title="Cluster Visualization",
    plot_bgcolor="white",
    width=700,
    height=600,
)

color = ["red", "green", "cyan", "magenta"]

for i in range(k):
    fig.add_trace(
        go.Scatter(
            x=groups[i][:, 0],
            y=groups[i][:, 3],
            mode="markers",
            marker=dict(size=5, color=color[i]),
            name="Points",
        )
    )

fig.add_trace(
    go.Scatter(
        x=centers[:, 0],
        y=centers[:, 3],
        mode="markers",
        marker=dict(size=10, color=color[3]),
        name="Points",
    )
)

fig.show()

In [29]:
centers[:, :]

array([[ 33.01538462,  37.86153846,  31.38461538,  50.15384615,
          0.4       ],
       [167.5       ,  36.28787879,  89.42424242,  50.13636364,
          0.46969697],
       [ 99.98550725,  42.23188406,  60.43478261,  50.30434783,
          0.44927536]])

In [58]:
df_score = []

In [ ]:
sil = silhouette_score(df_edited, lables)
print("Silhouette:", sil)
ch = calinski_harabasz_score(df_edited, lables)
print("Calinski-Harabasz:", ch)
db = davies_bouldin_score(df_edited, lables)
print("Davies-Bouldin:", db)

df_score.append(["K-means", sil, ch, db])

Silhouette: 0.3707664002450904
Calinski-Harabasz: 255.3422080058244
Davies-Bouldin: 0.9072034871161095


In [61]:
df_score

[['K-means', 0.3707664002450904, 255.3422080058244, 0.9072034871161095]]

# Иерархические методы

In [ ]:
from sklearn.cluster import AgglomerativeClustering


k = 3
hier = AgglomerativeClustering(n_clusters=k, linkage="ward")

hier_labels = hier.fit_predict(df_edited)

In [63]:
groups = [df_edited[np.where(hier_labels == i)] for i in range(k)]

In [64]:
centers = np.vstack([group.mean(axis=0) for group in groups])

In [ ]:
fig = go.Figure()

fig.update_layout(plot_bgcolor="white", width=700, height=600)


for i in range(k):
    fig.add_trace(
        go.Scatter(
            x=groups[i][:, 0],
            y=groups[i][:, 3],
            mode="markers",
            marker=dict(size=5, color=color[i]),
            name=f"Cluster {i+1}",
        )
    )

fig.add_trace(
    go.Scatter(
        x=centers[:, 0],
        y=centers[:, 3],
        mode="markers",
        marker=dict(size=10, color=color[3]),
        name="Centers",
    )
)

fig.show()

In [ ]:
sil = silhouette_score(df_edited, hier_labels)
print("Silhouette:", sil)
ch = calinski_harabasz_score(df_edited, hier_labels)
print("Calinski-Harabasz:", ch)
db = davies_bouldin_score(df_edited, hier_labels)
print("Davies-Bouldin:", db)

df_score.append(["AgglomerativeClustering", sil, ch, db])

Silhouette: 0.33073425793530475
Calinski-Harabasz: 236.31342567534764
Davies-Bouldin: 0.9222140127612253


In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import StandardScaler
import plotly.figure_factory as ff


X = df_edited

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Z = linkage(X_scaled, method="ward")

fig = ff.create_dendrogram(X_scaled, linkagefun=lambda x: Z)
fig.update_layout(
    width=900,
    height=500,
)
fig.show()

In [ ]:
labels = fcluster(Z, t=k, criterion="maxclust")

In [69]:
labels

array([2, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 2, 3, 2, 3, 2, 2, 2,
       3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2,
       3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 2,
       3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2,
       2, 3, 3, 2, 3, 3, 2, 2, 3, 2, 3, 2, 2, 3, 3, 2, 3, 2, 3, 3, 3, 3,
       3, 2, 3, 2, 2, 2, 3, 3, 3, 3, 2, 3, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int32)

In [70]:
labels -= 1
labels

array([1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1,
       2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1,
       2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1,
       1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2,
       2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)

In [71]:
groups = [df_edited[np.where(labels == i)] for i in range(k)]
centers = np.vstack([group.mean(axis=0) for group in groups])

In [ ]:
fig = go.Figure()

fig.update_layout(plot_bgcolor="white", width=700, height=600)


for i in range(k):
    fig.add_trace(
        go.Scatter(
            x=groups[i][:, 0],
            y=groups[i][:, 3],
            mode="markers",
            marker=dict(size=5, color=color[i]),
            name=f"Cluster {i+1}",
        )
    )

fig.add_trace(
    go.Scatter(
        x=centers[:, 0],
        y=centers[:, 3],
        mode="markers",
        marker=dict(size=10, color=color[3]),
        name="Centers",
    )
)

fig.show()

In [ ]:
sil = silhouette_score(df_edited, labels)
print("Silhouette:", sil)
ch = calinski_harabasz_score(df_edited, labels)
print("Calinski-Harabasz:", ch)
db = davies_bouldin_score(df_edited, labels)
print("Davies-Bouldin:", db)

df_score.append(["Dendrogramm", sil, ch, db])

Silhouette: 0.2783001964834928
Calinski-Harabasz: 146.29290197643994
Davies-Bouldin: 1.790811077803064


In [ ]:
cols = ["Method", "Silhouette", "Calinski-Harabasz", "Davies-Bouldin"]
df = pd.DataFrame(df_score, columns=cols)

In [76]:
df

,Method,Silhouette,Calinski-Harabasz,Davies-Bouldin
0,K-means,0.370766,255.342208,0.907203
1,AgglomerativeClustering,0.330734,236.313426,0.922214
2,Dendrogramm,0.278300,146.292902,1.790811
